# Contacts-0a: Queries

In dit notebook gaan verder we in op MongoDB-queries.

We beginnen met de standaardacties voor het importeren van de belangrijkste libraries, om verbinding te maken met de database, en om de voorbeeld-data in te lezen.

> Het resultaat van de laatste opdracht (mongoimport) moet 0 zijn, anders is er sprake van een probleem. Dit moet je eerst (laten) oplossen: doorgaan met de rest heeft dan geen zin.

## Initialisaties

In [ ]:
import os
import re
import pandas as pd
import numpy as np
from IPython.core.display import display, HTML
import pymongo

pd.set_option('max_colwidth',160)

userline = !echo $USER
username = userline[0]
dbname = username + "-demodb"
print("Database name: " + dbname)

from pymongo import MongoClient
print('Mongo version', pymongo.__version__)
client = MongoClient('localhost', 27017)
db = client[dbname]
collection = db.contacts

collection.drop()
os.system('/usr/local/bin/mongoimport -d ' + dbname + ' -c contacts adressen.json')

## Query: alle elementen

Als controle op de database zoeken we alle elementen.
(Voor een realistische database is dat ondoenlijk.)

In [ ]:
cursor = collection.find()
for obj in cursor:
    print(obj)

## Query: zoeken op naam

De volgende opdracht is om de gegevens van een of meerdere personen te zoeken,
op basis van een deel van de gegevens.
We geven in de zoekopdracht een gedeeltelijk ingevuld document;
de zoekopdracht vindt dan alle documenten die met die invulling overeenkomen.

> Opmerking: dit komt overeen met het `where`-deel in een SQL-query.

In [ ]:
cursor1 = collection.find({"name": "Anna Verschuur"})
for obj in cursor1:
    print(obj)

In [ ]:
cursor2 = collection.find({"address.city": "Amsterdam"})
for obj in cursor2:
    print(obj)

**Opdracht** maak hieronder een query voor het zoeken van alle inwoners van Rotterdam in de database.

## Find: reguliere expressies

Met behulp van reguliere expressies kun je zoeken op waarden die voor een deel bepaald zijn. In het voorbeeld zoeken we op namen waar `schuur` in voorkomt.

> MongoDB/pymongo gebruikt hiervoor de Python reguliere expressies, zie: https://docs.python.org/3/library/re.html en https://docs.python.org/3/howto/regex.html.

> In het geval van SQL gebruik je hiervoor de LIKE constructie.

In [ ]:
cursor = collection.find({"name": re.compile("schuur")})
list(cursor)

**Opdracht** Zoek personen die in Amsterdam of in Rotterdam wonen (door een handige reguliere expressie te kiezen).

## Zoeken naar één document

Voor het zoeken naar één enkel document gebruik je: `find_one`.
Als er geen document gevonden wordt is het resultaat `None`.
(Dit is *de lege waarde* in Python.)

In [ ]:
anna = collection.find_one({"name": "Anna Verschuur"})
anna

## Samengestelde queries

Als je meerdere eigenschappen gebruikt in een query, dan heeft dit de betekenis van een "and":
de gezochte documenten voldoen aan alle deel-eisen.

Soms heb je andere samenstellingen nodig, bijvoorbeeld als je zoekt naar personen in Amsterdam of in Rotterdam.
In dit geval gebruik je een `$or` met een lijst van deel-eisen.

In [ ]:
cursor5 = collection.find(
    {"$or": [{"address.city": "Rotterdam"}, {"address.city": "Amsterdam"}]})
list(cursor5)

## Projectie

In de voorbeelden hierboven hebben we de complete documenten laten zien.
Vaak zijn we maar in bepaalde onderdelen geïnteresseerd.
In een *projectie* geven we de gewenste onderdelen van het resultaat aan.
Zo'n projectie bevat de velden van het document die we willen zien, met een "1" als waarde.
We kunnen ook aangeven welke velden we willen weglaten (exclusie): die geven we aan met een "0" als waarde.
Dit gebruiken we bijvoorbeeld als we de *key* (`_id`) niet nodig hebben.

> Opmerking: projectie komt overeen met het SELECT-deel in een SQL-query

In [ ]:
projection3 = {"_id":0, "name": 1, "address": 1}
list3 = list(collection.find({"address.city": "Amsterdam"}, projection3))
list3

We kunnen een dergelijke lijst ook als (pandas) tabel weergeven.
Daarbij kunnen we ook de volgorde van de elementen aangeven.
Let hierbij op de dubbele `[[ ]]`: dit is ook een *projectie*, maar nu in Python/pandas.

> Het is niet netjes om het geneste document "address" zo te laten staan, maar dat laten we nu even voor wat het is.

In [ ]:
df0 = pd.DataFrame(list3)
display(df0[["name", "address"]])

## Sorteren

We kunnen de volgorde van de documenten in het resultaat ook aanpassen, met behulp van `sort`.
We geven daarbij het veld en de sorteerrichting aan.

In [ ]:
cursor4 = collection.find({"address.city": "Amsterdam"}, {"_id":0, "name": 1, "address": 1})
cursor4.sort("name", pymongo.ASCENDING)
list(cursor4)

## Alleen documenten met een bepaald veld

In plaats van de *waarde* van een veld, kunnen we ook het predicaat-waarde `$exists` opgeven: we vinden dan alleen die documenten die een waarde hebben voor dit veld.
Dit is in MongoDB een zinvolle query, omdat documenten verschillend kunnen zijn van structuur.

> In SQL heeft een rij altijd alle velden, maar een veld kan wel *leeg* (NULL) zijn. Dat komt het meest in de buurt van deze vorm.
    

In [ ]:
projection4 = {"_id":0, "name": 1, "email": 1, "tel": 1}
cursor4 = collection.find({"email": {"$exists": True}}, projection4)
list(cursor4)

## Documenten met een ontbrekend veld

Het tegenovergestelde is ook vaak nuttig: in welke documenten ontbreekt een bepaald veld? (Via: `"$exists": False`.)

In [ ]:
cursor5 = collection.find({"address": {"$exists": True}, "address.postcode": {"$exists": False}})
list(cursor5)

## Welke waarden?

Soms wil je weten welke waarden voor een bepaald veld in de documenten in de database voorkomt.
Bijvoorbeeld: welke plaatsen komen voor in de database?
Met behulp van `distinct` kun je dit opvragen.

In [ ]:
collection.distinct("address.city")